In [149]:
#Imports 
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import math
import cv2 #Load Images 
import os  #For files and folder management 
from tqdm import tqdm #to trace iterations during model training

# loading images

In [150]:
!git clone https://github.com/laxmimerit/dog-cat-full-dataset.git #data

fatal: destination path 'dog-cat-full-dataset' already exists and is not an empty directory.


In [151]:
#get list of files' full paths using os model
cat_folder = '/content/dog-cat-full-dataset/data/train/cats'
dog_folder = '/content/dog-cat-full-dataset/data/train/dogs'

cat_files=[cat_folder+"/"+i for i in os.listdir(cat_folder)]
dog_files=[dog_folder+"/"+i for i in os.listdir(dog_folder)]
print (cat_files)

['/content/dog-cat-full-dataset/data/train/cats/cat.5380.jpg', '/content/dog-cat-full-dataset/data/train/cats/cat.3400.jpg', '/content/dog-cat-full-dataset/data/train/cats/cat.2991.jpg', '/content/dog-cat-full-dataset/data/train/cats/cat.894.jpg', '/content/dog-cat-full-dataset/data/train/cats/cat.9480.jpg', '/content/dog-cat-full-dataset/data/train/cats/cat.5961.jpg', '/content/dog-cat-full-dataset/data/train/cats/cat.9065.jpg', '/content/dog-cat-full-dataset/data/train/cats/cat.6657.jpg', '/content/dog-cat-full-dataset/data/train/cats/cat.1405.jpg', '/content/dog-cat-full-dataset/data/train/cats/cat.5142.jpg', '/content/dog-cat-full-dataset/data/train/cats/cat.10793.jpg', '/content/dog-cat-full-dataset/data/train/cats/cat.12409.jpg', '/content/dog-cat-full-dataset/data/train/cats/cat.320.jpg', '/content/dog-cat-full-dataset/data/train/cats/cat.7613.jpg', '/content/dog-cat-full-dataset/data/train/cats/cat.37.jpg', '/content/dog-cat-full-dataset/data/train/cats/cat.4896.jpg', '/content

# Labeled dataset

In [152]:
cat_labeled=[[0,i]for i in cat_files]
dog_labeled=[[1,i]for i in dog_files]
labeled_data=cat_labeled+ dog_labeled

random.shuffle(labeled_data)

# Data Pipelines With Generator

In [153]:
def DataPipeLine(data,batch_size=48,width=64, height=64):
  #flaten photos into 1d array 
  imgs_arr=[]
  labels=[]
  dim = (width, height)
  batch_n=int((len(data))/batch_size)
  for d in range(batch_n):
    # imgs_arr =np.array([cv2.imread(j).flatten() for [i,j] in data[d*batch_size:(batch_size*(d+1))+1]]) #read the img change it into 1d arr
    
    labels   =np.array([ i for [i,j] in data[d*batch_size:(batch_size*(d+1))]]).reshape(batch_size,1)
    #read the img change it into 1d arr and resize it devided by 255 for normalization
    imgs_arr =np.array([ (( cv2.resize(cv2.imread(j),dim).flatten() )/255) for [i,j] in data[d*batch_size:(batch_size*(d+1))]])
    yield imgs_arr,labels



# Logistic Regression loss function

In [157]:
#Log Loss
def CostFunction (Predictions,GroundTruth): 
  n=len(GroundTruth)
  log_loss =(-1/n)*np.sum( (GroundTruth.T* np.log(Predictions)) +( (1-GroundTruth).T*np.log(1-Predictions) ))
  return log_loss




# Optimization Gradient Descent 

In [158]:
#activation function to map predections into 1 and zero probability  
def sigmoid(predections):
  A = 1/(1+np.exp(-predections))
  
  for i,elem in enumerate(A):
    if (elem <= 1.01086754e-02 ): A[i][0] = 1.01086754e-02
    elif (elem == 1): A[i][0] = 0.99 

  assert A.dtype == np.float64
  return A

In [159]:
def GradientDescent(labeled_data,iterations,batch_size=48,width=64, height=64):
  w = np.full_like(np.arange(height*width*3), 0.05, dtype=np.float).reshape(((height*width*3),1))
  w0= np.zeros((batch_size,1))
  # print (w0.shape,w.shape)
  batch_n = int(len(labeled_data)/batch_size)
  for i in tqdm (range(iterations)):
    cost_li = []
    img=DataPipeLine(labeled_data)
    for ii in range(batch_n):
      x,GroundTruth=next(img)
      predictions=sigmoid(np.dot(x,w))
      #derivative for optmaization 
      d_w0= (predictions-GroundTruth)
      d_w = np.dot(x.T,(predictions-GroundTruth))
      #updating weights
      w0 = w0-(0.0001)*(1/batch_size)*d_w0
      w  = w -(0.0001)*(1/batch_size)*d_w
      cost_li.append(CostFunction(predictions,GroundTruth))
    print (sum(cost_li)/batch_n,"arv patch loss \n")
  return w0 , w , cost_li

In [ ]:
w0 , w , cost_li=GradientDescent(labeled_data,30)
print(w0 , w , cost_li)

  0%|          | 0/30 [00:00<?, ?it/s]